In [ ]:
import time
from collections import deque
import heapq
import ipywidgets as widgets
from IPython.display import display, clear_output
import matplotlib.pyplot as plt

In [ ]:
# 2. Representação do Estado e Ações

class EstadoPuzzle:
    """
    Representa um estado do tabuleiro 3x3 do Jogo dos 8.
    """
    def __init__(self, tabuleiro, vazio=0):
        self.tabuleiro = tabuleiro
        self.vazio = vazio
        self.pos_vazio = self._encontrar_posicao(vazio)

    def _encontrar_posicao(self, peca):
        """Encontra a posição (linha, coluna) de uma peça no tabuleiro."""
        for r, linha in enumerate(self.tabuleiro):
            for c, valor in enumerate(linha):
                if valor == peca:
                    return (r, c)
        return None

    def __eq__(self, other):
        """Verifica se dois estados são iguais."""
        return self.tabuleiro == other.tabuleiro

    def __hash__(self):
        """Permite que o estado seja usado em um set ou dict."""
        return hash(self.tabuleiro)

    def __str__(self):
        """Retorna uma representação em string formatada do tabuleiro."""
        s = ""
        for linha in self.tabuleiro:
            s += " ".join(str(x).replace('0', '_') for x in linha) + "\n"
        return s

    def gerar_sucessores(self):
        """Gera todos os estados sucessores válidos (movimentos possíveis)."""
        sucessores = []
        r, c = self.pos_vazio
        movimentos = [
            (-1, 0, 'Cima'),  (1, 0, 'Baixo'),
            (0, -1, 'Esquerda'), (0, 1, 'Direita')
        ]
        for dr, dc, acao in movimentos:
            nova_r, nova_c = r + dr, c + dc
            if 0 <= nova_r < 3 and 0 <= nova_c < 3:
                novo_tabuleiro_lista = [list(linha) for linha in self.tabuleiro]
                novo_tabuleiro_lista[r][c] = novo_tabuleiro_lista[nova_r][nova_c]
                novo_tabuleiro_lista[nova_r][nova_c] = self.vazio
                novo_tabuleiro_tuple = tuple(tuple(linha) for linha in novo_tabuleiro_lista)
                sucessores.append((acao, EstadoPuzzle(novo_tabuleiro_tuple)))
        return sucessores

In [ ]:
# 3. O Nó da Busca - retorna os estados

class NoBusca:
    """
    Representa um nó na árvore de busca.
    """
    def __init__(self, estado, pai=None, acao=None, g=0):
        self.estado = estado
        self.pai = pai
        self.acao = acao
        self.g = g

    def __lt__(self, other):
        """Comparador 'menor que', necessário para a fila de prioridade (heapq)."""
        return self.g < other.g

    def reconstruir_caminho(self):
        """Retorna a sequência de AÇÕES da raiz até este nó."""
        caminho = []
        no_atual = self
        while no_atual.pai is not None:
            caminho.append(no_atual.acao)
            no_atual = no_atual.pai
        return list(reversed(caminho))

    def reconstruir_caminho_de_estados(self):
        """Retorna a sequência de ESTADOS da raiz até este nó."""
        estados = []
        no_atual = self
        while no_atual is not None:
            estados.append(no_atual.estado)
            no_atual = no_atual.pai
        return list(reversed(estados))

In [ ]:
# 4. Algoritmo 1: Busca em Amplitude (BFS)

def bfs(estado_inicial, estado_objetivo):
    """
    Executa a Busca em Amplitude (BFS).
    Retorna (no_final, nos_visitados) ou (None, nos_visitados)
    """
    print("Iniciando Busca em Amplitude (BFS)...")
    fronteira = deque()
    visitados = set()
    no_raiz = NoBusca(estado_inicial, g=0)
    fronteira.append(no_raiz)
    visitados.add(no_raiz.estado)

    while fronteira:
        no_atual = fronteira.popleft()
        if no_atual.estado == estado_objetivo:
            print("Solução encontrada!")
            return no_atual, len(visitados)
        for acao, proximo_estado in no_atual.estado.gerar_sucessores():
            if proximo_estado not in visitados:
                visitados.add(proximo_estado)
                novo_no = NoBusca(
                    estado=proximo_estado, pai=no_atual,
                    acao=acao, g=no_atual.g + 1
                )
                fronteira.append(novo_no)
    print("Solução não encontrada.")
    return None, len(visitados)

In [ ]:
# 5. Algoritmo 2: Busca A*

# 5.1. Heurística: Distância de Manhattan
def calcular_posicoes_objetivo(estado_objetivo):
    """Cria um dicionário mapeando cada peça à sua posição no objetivo."""
    posicoes = {}
    for r, linha in enumerate(estado_objetivo.tabuleiro):
        for c, peca in enumerate(linha):
            if peca != estado_objetivo.vazio:
                posicoes[peca] = (r, c)
    return posicoes

def h_manhattan(estado, posicoes_objetivo, vazio=0):
    """Calcula a heurística da Distância de Manhattan."""
    distancia_total = 0
    for r, linha in enumerate(estado.tabuleiro):
        for c, peca in enumerate(linha):
            if peca != vazio:
                pos_obj_r, pos_obj_c = posicoes_objetivo[peca]
                distancia_total += abs(r - pos_obj_r) + abs(c - pos_obj_c)
    return distancia_total

#  5.2. Implementação do A*
def a_estrela(estado_inicial, estado_objetivo):
    """
    Executa a Busca A* (A-Star).
    Retorna (no_final, nos_visitados) ou (None, nos_visitados)
    """
    print("\nIniciando Busca A*...")
    pos_objetivo = calcular_posicoes_objetivo(estado_objetivo)
    fronteira = []
    visitados = set()
    no_raiz = NoBusca(estado_inicial, g=0)
    h_raiz = h_manhattan(no_raiz.estado, pos_objetivo)
    f_raiz = no_raiz.g + h_raiz
    heapq.heappush(fronteira, (f_raiz, no_raiz))
    custo_g = {estado_inicial: 0}

    while fronteira:
        f_atual, no_atual = heapq.heappop(fronteira)
        if no_atual.estado == estado_objetivo:
            print("Solução encontrada!")
            return no_atual, len(visitados)
        if no_atual.estado in visitados:
            continue
        visitados.add(no_atual.estado)

        for acao, proximo_estado in no_atual.estado.gerar_sucessores():
            novo_g = no_atual.g + 1
            if proximo_estado in custo_g and novo_g >= custo_g[proximo_estado]:
                continue
            custo_g[proximo_estado] = novo_g
            h_novo = h_manhattan(proximo_estado, pos_objetivo)
            f_novo = novo_g + h_novo
            novo_no = NoBusca(
                estado=proximo_estado, pai=no_atual,
                acao=acao, g=novo_g
            )
            if proximo_estado not in visitados:
                 heapq.heappush(fronteira, (f_novo, novo_no))
    print("Solução não encontrada.")
    return None, len(visitados)

In [ ]:
# 6. Interface Interativa e Execução (com Visualizador)

#  Definição do Estado Objetivo (Fixo)
OBJETIVO_TUPLE = (
    (1, 2, 3),
    (8, 0, 4),
    (7, 6, 5)
)
estado_objetivo = EstadoPuzzle(OBJETIVO_TUPLE)

#  Criação dos Widgets da Interface Inicial
titulo = widgets.HTML("<h2>Jogo dos 8</h2>")
header = widgets.HTML("<h3>Defina o Estado Inicial do Tabuleiro (use 0 para o vazio):</h3>")

# Valores padrão (do exemplo da imagem)
valores_iniciais = ['2', '0', '3', '1', '7', '4', '6', '8', '5']

# 9 caixas de texto para o grid 3x3
text_inputs = [
    widgets.Text(value=val, layout=widgets.Layout(width='50px', height='50px',
                                                  display='flex', justify_content='center', align_items='center'))
    for val in valores_iniciais
]

# Organiza as caixas de texto em um grid
grid_inicial = widgets.GridBox(
    children=text_inputs,
    layout=widgets.Layout(
        width='240px',
        grid_template_columns='60px 60px 60px',
        grid_template_rows='60px 60px 60px',
        grid_gap='5px'
    )
)

# Botão para iniciar a busca
run_button = widgets.Button(
    description='Resolver',
    button_style='success',
    tooltip='Iniciar buscas BFS e A*'
)

# Widget de saída principal
output_area = widgets.Output()

# Mostra o estado objetivo para referência
objetivo_header = widgets.HTML(f"<h4>Estado Objetivo (Fixo):</h4><pre style='font-size: 1.2em; line-height: 1.4;'>{str(estado_objetivo)}</pre>")


#  Lógica de Execução (Função do Botão Principal)
def validar_e_rodar(b):
    """
    Função chamada quando o botão 'Resolver' é clicado.
    Valida o input, executa as buscas e exibe o visualizador.
    """
    with output_area:
        clear_output(wait=True)
        print("\nIniciando validação do tabuleiro...\n")

        # (Validação do grid)
        numeros = set()
        valores_grid = []
        try:
            for i in range(9):
                val_str = text_inputs[i].value.strip()
                if not val_str: raise ValueError(f"Posição ({i//3}, {i%3}) está vazia.")
                val = int(val_str)
                if not (0 <= val <= 8): raise ValueError("Valores devem estar entre 0 e 8.")
                if val in numeros: raise ValueError(f"Valor duplicado encontrado: {val}")
                numeros.add(val)
                valores_grid.append(val)
        except ValueError as e:
            print(f"Erro no Input: {e}")
            print("Por favor, insira números únicos de 0 a 8.")
            return

        print("Validação OK.\n")

        INICIAL_TUPLE = (
            tuple(valores_grid[0:3]),
            tuple(valores_grid[3:6]),
            tuple(valores_grid[6:9])
        )
        estado_inicial = EstadoPuzzle(INICIAL_TUPLE)

        print("Estado Inicial:")
        print(estado_inicial)

        if estado_inicial == estado_objetivo:
            print("✨ O tabuleiro inicial já está no estado objetivo! ✨")
            print("\nComparação")
            print("Custo: 0 passos.")
            print("Nós visitados: 0 (nenhuma busca foi executada).")
            return

        #  Executar BFS
        start_time_bfs = time.time()
        no_final_bfs, visitados_bfs = bfs(estado_inicial, estado_objetivo)
        end_time_bfs = time.time()

        if no_final_bfs is not None:
            custo_bfs = no_final_bfs.g
            caminho_bfs = no_final_bfs.reconstruir_caminho()
            print(f"  Custo (passos): {custo_bfs}")
            print(f"  Caminho: {caminho_bfs}")
            print(f"  Nós visitados: {visitados_bfs}")
            print(f"  Tempo de execução: {end_time_bfs - start_time_bfs:.4f} segundos")
        else:
            print("  BFS: Solução não encontrada.")

        #  Executar A*
        start_time_astar = time.time()
        no_final_astar, visitados_astar = a_estrela(estado_inicial, estado_objetivo)
        end_time_astar = time.time()

        if no_final_astar is not None:
            custo_astar = no_final_astar.g
            caminho_astar = no_final_astar.reconstruir_caminho()
            print(f"  Custo (passos): {custo_astar}")
            print(f"  Caminho: {caminho_astar}")
            print(f"  Nós visitados: {visitados_astar}")
            print(f"  Tempo de execução: {end_time_astar - start_time_astar:.4f} segundos")
            print("A execução passo-a-passo do A* está logo abaixo da seção de Comparação!")
        else:
            print("  A*: Solução não encontrada.")

        # Comparação
        print("\nComparação")
        bfs_time = end_time_bfs - start_time_bfs
        astar_time = end_time_astar - start_time_astar
        custo_bfs_str = no_final_bfs.g if no_final_bfs else 'N/A'
        custo_astar_str = no_final_astar.g if no_final_astar else 'N/A'

        print(f"BFS: {custo_bfs_str} passos, {visitados_bfs} nós visitados, {bfs_time:.4f}s")
        print(f"A*:  {custo_astar_str} passos, {visitados_astar} nós visitados, {astar_time:.4f}s")

        #graficos de comparação
        try:
            print("\nGráficos de Comparação:")
            # Garante que os valores de tempo sejam positivos para a escala log
            min_time = 1e-6
            times_to_plot = [max(bfs_time, min_time), max(astar_time, min_time)]
            nodes_to_plot = [visitados_bfs, visitados_astar]
            labels = ['BFS', 'A*']
            colors = ['#ff6347', '#4682b4'] # Vermelho, Azul

            # 1. Gráfico de Tempo
            plt.figure(figsize=(4, 4))
            bars = plt.bar(labels, times_to_plot, color=colors)
            plt.ylabel('Tempo de Execução (segundos)')
            plt.title('Comparação de Tempo de Execução (Escala Logarítmica)')
            plt.yscale('log') # Usar escala logarítmica é essencial
            plt.bar_label(bars, fmt='%.4fs')
            plt.tight_layout()
            plt.show() # Exibe o gráfico na saída

            # 2. Gráfico de Nós Visitados
            plt.figure(figsize=(5, 4))
            bars = plt.bar(labels, nodes_to_plot, color=colors)
            plt.ylabel('Nós Visitados')
            plt.title('Comparação de Nós Visitados (Escala Logarítmica)')
            plt.yscale('log') # Escala logarítmica é OBRIGATÓRIA aqui
            plt.bar_label(bars, fmt='%d')
            plt.tight_layout()
            plt.show() # Exibe o gráfico na saída

        except Exception as e:
            print(f"Erro ao gerar gráficos: {e}")

        #  VISUALIZADOR PASSO-A-PASSO (A*)
        if no_final_astar:
            print("\n Visualizador da Solução (A*) ")

            # 1. Obter a lista de estados da solução
            estados_do_caminho = no_final_astar.reconstruir_caminho_de_estados()
            total_passos = len(estados_do_caminho) - 1

            # 2. Dicionário para guardar o passo atual
            visualizer_state = {'passo': 0}

            # 3. Criar os widgets do visualizador
            layout_visualizer = widgets.Layout(width='50px', height='50px', display='flex', justify_content='center', align_items='center')
            visualizer_inputs = [
                widgets.Text(value='', layout=layout_visualizer, disabled=True,
                             style={'text_color': 'black', 'font_weight': 'bold'})
                for _ in range(9)
            ]
            visualizer_grid = widgets.GridBox(
                children=visualizer_inputs,
                layout=widgets.Layout(
                    width='240px', grid_template_columns='60px 60px 60px',
                    grid_template_rows='60px 60px 60px', grid_gap='5px'
                )
            )
            prev_button = widgets.Button(description="<< Anterior")
            next_button = widgets.Button(description="Próximo >>")
            step_label = widgets.HTML(value=f"Passo: 0 / {total_passos}")
            controls = widgets.HBox(
                [prev_button, step_label, next_button],
                layout=widgets.Layout(display='flex', justify_content='space-between', width='250px', margin='10px 0 0 -35px')
            )

            # 4. Função para atualizar o grid do visualizador
            def update_visualizer(passo):
                estado_atual = estados_do_caminho[passo]
                k = 0
                for r in range(3):
                    for c in range(3):
                        val = estado_atual.tabuleiro[r][c]
                        visualizer_inputs[k].value = str(val).replace('0', '_')
                        k += 1
                step_label.value = f"<b>Passo: {passo} / {total_passos}</b>"
                prev_button.disabled = (passo == 0)
                next_button.disabled = (passo == total_passos)

            # 5. Funções de callback dos botões
            def on_prev_click(b):
                if visualizer_state['passo'] > 0:
                    visualizer_state['passo'] -= 1
                    update_visualizer(visualizer_state['passo'])

            def on_next_click(b):
                if visualizer_state['passo'] < total_passos:
                    visualizer_state['passo'] += 1
                    update_visualizer(visualizer_state['passo'])

            prev_button.on_click(on_prev_click)
            next_button.on_click(on_next_click)

            # 6. Inicializa o visualizador no passo 0
            update_visualizer(0)

            # 7. Exibe Grid + Controles
            display(widgets.VBox(
                [visualizer_grid, controls],
                layout=widgets.Layout(display='flex', align_items='center') # Centraliza
            ))
        #  fim do visualizador


# "Linka" a função principal ao clique do botão
run_button.on_click(validar_e_rodar)

#  Montagem Final da interface, exibe os componentes iniciais
display(titulo, header, grid_inicial, run_button, objetivo_header, output_area)